In [ ]:
# Paths - Update locally!
git_path = '/path/to/git/kurteff2024_code/'
data_path = '/path/to/bids/dataset/'

In [ ]:
import mne
import numpy as np
import pandas as pd
import os
import re
import csv
from tqdm.notebook import tqdm
import warnings

from img_pipe import img_pipe

from matplotlib import pyplot as plt
from matplotlib import rcParams as rc
import matplotlib.patheffects as PathEffects
rc['pdf.fonttype'] = 42
plt.style.use('seaborn')
%matplotlib inline

In [ ]:
subjs = [s for s in os.listdir(
    os.path.join(git_path,"preprocessing","events","csv")) if "TCH" in s or "S0" in s]
exclude = ["TCH8"]
no_imaging = ["S0010"]
subjs = [s for s in subjs if s not in exclude]

blocks = {
    s: [
        b.split("_")[-1] for b in os.listdir(os.path.join(
            git_path,"analysis","events","csv",s)) if f"{s}_B" in b and os.path.isfile(os.path.join(
            git_path,"analysis","events","csv",s,b,f"{b}_spkr_sn_all.txt"
        ))
    ] for s in subjs
}

smc_blocks = {
    s: [
        b.split("_")[-1] for b in os.listdir(os.path.join(
            git_path,"analysis","events","csv",s)) if f"{s}_B" in b and os.path.isfile(os.path.join(
            git_path,"analysis","events","csv",s,b,f"{b}_smc_mic.txt"
        ))
    ] for s in subjs
}

### Epoch the data

In [ ]:
baseline_tmin, baseline_tmax = -.5, 0. # Before the click
resp_tmin, resp_tmax = 0.5, 1. # After the click resp
baseline = None; epochs, ch_names = dict(), dict()
for s in tqdm([ss for ss in subjs if len(smc_blocks[ss])>0]):
    epochs[s] = dict(); epochs[s]['baseline'] = []; epochs[s]['resp'] = []
    for b in smc_blocks[s]:
        blockid = "_".join([s,b])
        raw = mne.io.read_raw_fif(os.path.join(data_path,f"sub-{s}",s,blockid,"HilbAA_70to150_8band",
                                               "ecog_hilbAA70to150.fif"), preload=True, verbose=False)
        fs = raw.info['sfreq']
        # Load event file
        ev_fpath = os.path.join(git_path,"preprocessing","events","csv",s,blockid,f"{blockid}_click_eve.txt")
        evs = np.loadtxt(ev_fpath)
        evs = np.vstack(((evs[:,0]*fs).astype(int),(evs[:,1]*fs).astype(int),evs[:,2].astype(int))).T
        epochs[s]['baseline'].append(mne.Epochs(raw, evs, tmin=baseline_tmin, tmax=baseline_tmax,
                                                baseline=baseline, preload=True, verbose=False))
        epochs[s]['resp'].append(mne.Epochs(raw, evs, tmin=resp_tmin, tmax=resp_tmax,
                                            baseline=baseline, preload=True, verbose=False))
    epochs[s]['baseline'] = mne.concatenate_epochs(epochs[s]['baseline'])
    epochs[s]['resp'] = mne.concatenate_epochs(epochs[s]['resp'])
    ch_names[s] = raw.info['ch_names']

### Run the bootstrap

In [ ]:
nboots = 1000; pvals = {s:{ch:0 for ch in ch_names[s]} for s in epochs.keys()}
for s in tqdm(epochs.keys()):
    baseline = epochs[s]['baseline'].get_data(); resp = epochs[s]['resp'].get_data()
    nepochs_baseline = baseline.shape[0]; nepochs_resp = resp.shape[0]
    for ch_idx, ch in enumerate(ch_names[s]):
        boot_results = []
        for n in np.arange(nboots):
            # Get a random ten epochs and average across these
            baseline_idx = np.random.choice(np.arange(nepochs_baseline), size=10)
            resp_idx = np.random.choice(np.arange(nepochs_resp), size=10)
            # Average resp across samples/epochs (chunk of 10)
            baseline_chunk = baseline[baseline_idx,ch_idx,:].mean(0).mean(0)
            resp_chunk = resp[resp_idx,ch_idx,:].mean(0).mean(0)
            if baseline_chunk > resp_chunk:
                boot_results.append(1)
            else:
                boot_results.append(0)
        pvals[s][ch] = sum(boot_results)/nboots

In [ ]:
# Save to dataframe
df = pd.DataFrame(columns = ['subj', 'ch_name', 'p'])
for s in pvals.keys():
    ch_names = list(pvals[s].keys())
    for ch in ch_names:
        new_row = pd.DataFrame({'subj':[s], 'ch_name':[ch], 'p':[pvals[s][ch]]})
        df = df.append(new_row, ignore_index=True)
df.to_csv(os.path.join(git_path,"stats","bootstraps","csv",
    f"seeg_elec_significance_{len(pvals.keys())}_subjs_{nboots}_boots_smc.csv"), index=False)  